In [26]:
import os
import numpy as np
from matplotlib import pyplot as plot
import math
from math import pi as pi
from astropy import units as u
#import panda as pd

In [28]:
#input variables - roughly based on those used in Lowell manuel
#example values for some of these variables can be found in the PDFs in the repository
# to do: input known values into tables (i.e. lunar brightness, airmass, etc)

SNR = 10
mag_raw = 30
exp_time = 0 * u.s

bandpass = "V" #possible values: UVBIR. Currently testing V
seeing = 2 * u.arcsec
binning = 2
lunar_age = 3 #example value, for V band, lunar age of 3 means 21.7 lunar brightness
airmass = 1

#possible other values:
psf = [0]
mag_norm = 20 #20 is what lowell uses
lunar_bright = 21.7 
signal_norm = 680 * u.electron / u.s / u.arcsec**2 #based on Lowell chart for V band

In [29]:
#calculate pixels

#if using seeing:
px = pi*(0.67*seeing)**2 * u.pix

#if using psf:
#px = pi*(0.67 * FWHM())

In [30]:
#Things governed by bandpass
#will  replace this with a dataframe with appropriate values

V = "V"
B = "B"
U = "U"
I = "I"
R = "R"

if bandpass.casefold() == V.casefold():
    extinct = 0.2
    
if bandpass.casefold() == B.casefold():
    extinct = 0.4
    
if bandpass.casefold() == U.casefold():
    extinct = 0.6
        
if bandpass.casefold() == I.casefold():
    extinct = 0.08
    
if bandpass.casefold() == R.casefold():
    extinct = 0.1
    
print(extinct)

0.2


In [43]:
#calculate sky noise
scale = .24 * u.arcsec / u.pix #this is defined by the telescope: http://www2.lowell.edu/rsch/LMI/specs.html
N_sky = scale**2 * binning**2 * signal_norm * 10**((-(lunar_bright - mag_norm))/2.5)  # Remove u.pix correction

print(N_sky)

32.73342034131643 electron / (pix2 s)


In [44]:
#calculate FWHM

In [45]:
#calculate signal from magnitude, with correction

mag_corr = mag_raw + extinct*airmass
signal = signal_norm * 10**((-(mag_corr - mag_norm))/2.5) * (u.electron / u.s / u.arcsec**2)

print(signal)

0.05655993643498162 electron2 / (arcsec4 s2)


In [48]:
#calculate SNR or exposure time or magnitude
#set Nron = 7, as seen in Lowell manuel
#ignore dark current
N_ron = 7 *(u.electron/u.s)

#SNR calculation:
def ETC_lowell_test(SNR, exp_time, mag_raw, signal, signal_norm, px, N_sky, N_ron, airmass, extinct):
    if SNR == 0:
        SNR = signal * exp_time/(math.sqrt(signal*exp_time + N_sky*exp_time*px + (N_ron**2)*px))
        mag_corr = mag_raw + extinct*airmass
        return SNR,exp_time,mag_corr
 
        
    if exp_time == 0:
        a = signal**2
        b = (SNR**2) * (signal * px*N_sky)
        c = (SNR**2) * px * N_ron
        exp_time = (-b + math.sqrt((b**2 - 4)*a*c))/(2*a)
        mag_corr = mag_raw + extinct*airmass
        return SNR,exp_time,mag_corr

    if mag_raw == 0:
        a = exp_time**2
        b = -exp_time * SNR
        c = (-SNR**2 * N_sky * px * exp_time) + (px * N_ron**2)
        signal = (-b + math.sqrt((b**2 - 4)*a*c))/(2*a)
        mag_raw = 2.5 * math.log((signal/signal_norm),10) + mag_norm
        mag_corr = mag_raw - (extinct*airmass)
        return SNR,exp_time,mag_corr



In [49]:
#print result

answer = ETC_lowell_test(SNR, exp_time, mag_raw, signal, signal_norm, px, N_sky, N_ron, airmass, extinct)
print("This is the SNR:", answer[0], "\n This is exposure time:", answer[1], "\n This is magnitude:", answer[2] )

if SNR <= 3:
    print("You have low SNR, cannot use this data")
if SNR < 10 and SNR >= 5:
    print("There's an object out there!...somewhere")
if SNR >= 10 and SNR < 90:
    print("You can use this data for measurement")
if SNR >= 90 and SNR <= 100:
    print("Your SNR is amazing!")
    

UnitConversionError: Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)